In [5]:
import torch
from wl_test_l2O import wl_test
from wl_test_final import WLtest
torch.set_printoptions(profile="full")

In [17]:
import gurobipy as gp
from gurobipy import GRB
import torch
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def graph_generator(ins_name):
    # Create a Gurobi model
    m = gp.read(ins_name)
    nvars = m.NumVars
    # Gurobi handles variables and constraints slightly differently
    mvars = m.getVars()
    # Sort variables by name (Gurobi does not guarantee order)
    mvars = sorted(mvars, key=lambda v: v.VarName)
    var_feature = torch.zeros((nvars, 2))

    
    # Obtain variable features
    for n in range(nvars):
        var = mvars[n]
        obj_coeff = var.Obj
        lb = var.LB
        ub = var.UB
        if ub != float('inf'):
            m.addConstr(var <= float('inf'))
        if lb != 0:
            m.addConstr(var >= 0)
        
        m.addConstr(var >= lb)
        m.addConstr(var <= ub)
        m.update()
        bin = 1 if var.VType == GRB.BINARY else 0
        feature = [obj_coeff, bin]
        var_feature[n] = torch.tensor(feature)

    # Obtain constraint features
    # for i in range(ncons):
        # cons = mcons[i]
        # sense = cons.Sense
        # rhs = cons.RHS
        # # lhs = -np.inf  # In Gurobi, the lower bound can be implicitly negative infinity
        # b = rhs
        # if sense == GRB.EQUAL:
            
        #     cons_sense = 0.0
        # elif sense == GRB.LESS_EQUAL:
            
        #     cons_sense = -1.0
        # elif sense == GRB.GREATER_EQUAL:
            
        #     cons_sense = 1.0
        # else:
        #     raise NotImplementedError('我只写了大于等于, 小于等于, 等于')
        
        # feature = [b, cons_sense]
        # cons_feature[i] = torch.tensor(feature)
    

    ncons = m.NumConstrs
    mcons = m.getConstrs()
    cons_feature = torch.zeros((ncons, 2))
    A = torch.zeros((ncons, nvars))
    for i in range(ncons):
        cons = mcons[i]
        sense = cons.Sense
        rhs = cons.RHS
        # lhs = -np.inf  # In Gurobi, the lower bound can be implicitly negative infinity
        b = rhs
        if sense == GRB.EQUAL:
            
            cons_sense = 0.0
        elif sense == GRB.LESS_EQUAL:
            
            cons_sense = -1.0
        elif sense == GRB.GREATER_EQUAL:
            
            cons_sense = 1.0
        else:
            raise NotImplementedError('我只写了大于等于, 小于等于, 等于')
        
        feature = [b, cons_sense]
        cons_feature[i] = torch.tensor(feature)
        for j in range(nvars):
            var = mvars[j]
            coeff = m.getCoeff(cons, var)  # Gurobi-specific function for constraint coefficients
            A[i, j] = coeff

    return A, var_feature, cons_feature

In [18]:
info1 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/answer_eoe_gpt-4o-2024-08-06.lp')
info2 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/model.lp')
A1 = info1[0]
A2 = info2[0]
f1 = info1[1]#variable
f2 = info2[1]
c1 = info1[2]#constrain
c2 = info2[2]
wltest = WLtest(A1, c1,f1,A2,c2,f2)
print(wltest.test())

Read LP format model from file /home/hanyizhou/wl_test/lp_generation/test/answer_eoe_gpt-4o-2024-08-06.lp
Reading time = 0.00 seconds
: 6 rows, 10 columns, 60 nonzeros
False
False
False
False
False
False
False
False
False
False
6
Read LP format model from file /home/hanyizhou/wl_test/lp_generation/test/model.lp
Reading time = 0.00 seconds
: 16 rows, 10 columns, 70 nonzeros
True
True
True
True
True
True
True
True
True
True
16
not same


In [8]:
c1.shape, c2.shape

(torch.Size([6, 2]), torch.Size([16, 2]))